### Models für Mimikanalyse : ResNet50 , Resnet101,  VGG16, VGG19, InceptionV3, DenseNet 169 und EfficientNetB7 mit Dataaugmenation

In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
import tensorflow as tf

print("TensorFlow-Version:", tf.__version__)

# Liste der verfügbaren Geräte abrufen
available_devices = tf.config.list_physical_devices()
print("Verfügbare Geräte:", available_devices)

# Überprüfen, ob eine GPU verfügbar ist
is_gpu_available = tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
print("Ist eine GPU verfügbar:", is_gpu_available)

# Spezifisch nach GPUs suchen
gpus = tf.config.list_physical_devices('GPU')
print("GPUs:", gpus)

TensorFlow-Version: 2.10.1
Verfügbare Geräte: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Ist eine GPU verfügbar: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
# TensorBoard Callback einrichten
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

#### ResNet50

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
import datetime

# Betitelung für Tensorboard
experiment_name = "ResNet50_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])

# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds)  

# Vortrainiertes Modell laden
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)  # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)

test_ds = preprocess_dataset(test_ds)  

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)


Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 30s 435ms/step - loss: 1.5086 - accuracy: 0.5386 - val_loss: 0.7096 - val_accuracy: 0.7206
Epoch 2/9
63/63 [==============================] - 21s 325ms/step - loss: 0.6216 - accuracy: 0.7673 - val_loss: 0.4523 - val_accuracy: 0.8423
Epoch 3/9
63/63 [==============================] - 13s 195ms/step - loss: 0.4767 - accuracy: 0.8201 - val_loss: 0.4029 - val_accuracy: 0.8603
Epoch 4/9
63/63 [==============================] - 13s 196ms/step - loss: 0.3973 - accuracy: 0.8475 - val_loss: 0.3964 - val_accuracy: 0.8363
Epoch 5/9
63/63 [==============================] - 13s 198ms/step - loss: 0.3561 - accuracy: 0.8665 - val_loss: 0.3048 - val_accuracy: 0.8802
Epoch 6/9
63/63 [==============================] - 13s 198ms/step - loss: 0.3230 - accuracy: 0.8705 - val_loss: 0.2463 - val_accuracy: 0.9182
Epo

#### ResNet101

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "ResNet101_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])

# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))
    
# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds) 

# Vortrainiertes Modell laden
base_model = ResNet101(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 24s 319ms/step - loss: 1.0965 - accuracy: 0.6378 - val_loss: 0.8675 - val_accuracy: 0.6307
Epoch 2/9
63/63 [==============================] - 21s 325ms/step - loss: 0.5178 - accuracy: 0.7912 - val_loss: 0.8431 - val_accuracy: 0.7046
Epoch 3/9
63/63 [==============================] - 20s 315ms/step - loss: 0.4192 - accuracy: 0.8396 - val_loss: 0.9053 - val_accuracy: 0.6447
Epoch 4/9
63/63 [==============================] - 19s 300ms/step - loss: 0.3325 - accuracy: 0.8744 - val_loss: 0.7333 - val_accuracy: 0.7006
Epoch 5/9
63/63 [==============================] - 21s 320ms/step - loss: 0.3085 - accuracy: 0.8809 - val_loss: 0.7338 - val_accuracy: 0.7066
Epoch 6/9
63/63 [==============================] - 21s 331ms/step - loss: 0.3455 - accuracy: 0.8620 - val_loss: 0.8798 - val_accuracy: 0.6387
Epo

#### VGG16

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "VGG16_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])

# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))
    
# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds) 

# Vortrainiertes Modell laden
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 32s 372ms/step - loss: 1.3330 - accuracy: 0.6034 - val_loss: 1.1860 - val_accuracy: 0.5689
Epoch 2/9
63/63 [==============================] - 17s 260ms/step - loss: 0.4712 - accuracy: 0.8166 - val_loss: 0.8247 - val_accuracy: 0.6747
Epoch 3/9
63/63 [==============================] - 17s 275ms/step - loss: 0.3504 - accuracy: 0.8655 - val_loss: 0.8557 - val_accuracy: 0.6906
Epoch 4/9
63/63 [==============================] - 18s 275ms/step - loss: 0.3099 - accuracy: 0.8844 - val_loss: 0.7495 - val_accuracy: 0.7186
Epoch 5/9
63/63 [==============================] - 17s 260ms/step - loss: 0.2575 - accuracy: 0.9033 - val_loss: 0.8939 - val_accuracy: 0.7186
Epoch 6/9
63/63 [==============================] - 17s 262ms/step - loss: 0.2243 - accuracy: 0.9153 - val_loss: 0.8539 - val_accuracy: 0.7146
Epo

#### VGG19

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "VGG19_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])

# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))
    
# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds) 

# Vortrainiertes Modell laden
base_model = VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x) # Dropout zur Regulierung von Overfitting
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für die Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 25s 335ms/step - loss: 1.2046 - accuracy: 0.6482 - val_loss: 0.9105 - val_accuracy: 0.6287
Epoch 2/9
63/63 [==============================] - 19s 294ms/step - loss: 0.4744 - accuracy: 0.8221 - val_loss: 0.9697 - val_accuracy: 0.6208
Epoch 3/9
63/63 [==============================] - 19s 299ms/step - loss: 0.3193 - accuracy: 0.8829 - val_loss: 0.8445 - val_accuracy: 0.6687
Epoch 4/9
63/63 [==============================] - 19s 290ms/step - loss: 0.2952 - accuracy: 0.8879 - val_loss: 0.9449 - val_accuracy: 0.6507
Epoch 5/9
63/63 [==============================] - 16s 249ms/step - loss: 0.2689 - accuracy: 0.8974 - val_loss: 0.9809 - val_accuracy: 0.5908
Epoch 6/9
63/63 [==============================] - 16s 251ms/step - loss: 0.2276 - accuracy: 0.9148 - val_loss: 0.9970 - val_accuracy: 0.5749
Epo

#### InceptionNetV3

In [19]:
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Betitelung für Tensorboard
experiment_name = "InceptionV3_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])

# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(299, 299),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(299, 299),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds) 

# Vortrainiertes Modell laden
base_model = InceptionV3(input_shape=(299, 299, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Einfrieren der Basis-Schichten


# Modell anpassen
inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(299, 299),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 115s 2s/step - loss: 1.1874 - accuracy: 0.5650 - val_loss: 1.3332 - val_accuracy: 0.5090
Epoch 2/9
63/63 [==============================] - 21s 327ms/step - loss: 0.7090 - accuracy: 0.7135 - val_loss: 1.1326 - val_accuracy: 0.5589
Epoch 3/9
63/63 [==============================] - 20s 318ms/step - loss: 0.5671 - accuracy: 0.7698 - val_loss: 1.0900 - val_accuracy: 0.5768
Epoch 4/9
63/63 [==============================] - 23s 361ms/step - loss: 0.4758 - accuracy: 0.8191 - val_loss: 1.2464 - val_accuracy: 0.5549
Epoch 5/9
63/63 [==============================] - 22s 349ms/step - loss: 0.4082 - accuracy: 0.8376 - val_loss: 1.4000 - val_accuracy: 0.5469
Epoch 6/9
63/63 [==============================] - 22s 352ms/step - loss: 0.4286 - accuracy: 0.8306 - val_loss: 1.5012 - val_accuracy: 0.5349
Epoch

#### DenseNet161

In [ ]:
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications import DenseNet169

# Betitelung für Tensorboard
experiment_name = "DenseNet169_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])

# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))    

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds) 

# Vortrainiertes Modell laden
base_model = DenseNet169(input_shape=(224, 224, 3), include_top=False, weights='imagenet')  # DenseNet 161
base_model.trainable = False  # Einfrieren der Basis-Schichten


# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für Ihre Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)

Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 19s 225ms/step - loss: 0.9123 - accuracy: 0.6796 - val_loss: 0.4090 - val_accuracy: 0.8663
Epoch 2/9
63/63 [==============================] - 12s 193ms/step - loss: 0.3216 - accuracy: 0.8869 - val_loss: 0.2384 - val_accuracy: 0.9202
Epoch 3/9
63/63 [==============================] - 13s 196ms/step - loss: 0.1850 - accuracy: 0.9472 - val_loss: 0.1891 - val_accuracy: 0.9281
Epoch 4/9
63/63 [==============================] - 12s 191ms/step - loss: 0.1469 - accuracy: 0.9492 - val_loss: 0.1676 - val_accuracy: 0.9381
Epoch 5/9
63/63 [==============================] - 13s 203ms/step - loss: 0.0954 - accuracy: 0.9741 - val_loss: 0.1240 - val_accuracy: 0.9581
Epoch 6/9
63/63 [==============================] - 12s 191ms/step - loss: 0.0932 - accuracy: 0.9726 - val_loss: 0.2523 - val_accuracy: 0.9022
Epo

#### EfficientNetB7

In [ ]:
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB7  # Update für EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import preprocess_input  # Update für EfficientNet-spezifische preprocess_input-Funktion

# Betitelung für Tensorboard
experiment_name = "EfficientNetB7_Dataaug"

# Data Augmentation Layer hinzufügen
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),  # Rotation um bis zu 20%
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),  # Horizontaler und vertikaler Flip
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),  # Zufällige Kontrastanpassung
])


# Funktion zur Vorbereitung und Augmentation der Bilder
def preprocess_and_augment_dataset(ds):
    # Zuerst Data Augmentation anwenden
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    # Dann die spezifische Vorverarbeitung für ResNet50 anwenden
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Funktion zur Vorbereitung der Testdaten
def preprocess_dataset(ds):
    return ds.map(lambda x, y: (preprocess_input(x), y))

# Trainingsdaten laden und vorbereiten
train_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung und Augmentation auf Trainingsdaten
train_ds = preprocess_and_augment_dataset(train_ds)

# Validierungsdaten laden und vorbereiten
val_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

# Anwendung der Vorverarbeitung (ohne Augmentation) auf Validierungsdaten
val_ds = preprocess_and_augment_dataset(val_ds) 

# Vortrainiertes Modell laden
base_model = EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet')  # Änderung zu EfficientNetB7
base_model.trainable = False  # Einfrieren der Basis-Schichten

# Modell anpassen
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(4, activation='softmax')(x)  # 4 Klassen für Ihre Emotionen
model = Model(inputs, outputs)

# Modell kompilieren und trainieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Setze den Pfad für die TensorBoard Logs
log_dir = "logs/fit/" + experiment_name + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, update_freq="epoch")

model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=9, 
    callbacks=[tensorboard_callback]  # TensorBoard Callback hinzufügen
)

# Modell mit Testdaten testen
test_ds = image_dataset_from_directory(
    "C:\\Thesis\\Data\\Frames\\Facial_Expressions_Test",
    image_size=(224, 224),
    batch_size=32)
test_ds = preprocess_dataset(test_ds)

# Modell evaluieren
test_loss, test_accuracy = model.evaluate(test_ds)

# Pfad für Test-Logs setzen
test_log_dir = "logs/fit/" + experiment_name + "_test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Summary Writer für Testdaten erstellen
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# Testmetriken in TensorBoard loggen
with test_summary_writer.as_default():
    tf.summary.scalar('test_loss', test_loss, step=0)
    tf.summary.scalar('test_accuracy', test_accuracy, step=0)


Found 2508 files belonging to 4 classes.
Using 2007 files for training.
Found 2508 files belonging to 4 classes.
Using 501 files for validation.
Epoch 1/9
63/63 [==============================] - 44s 570ms/step - loss: 0.6926 - accuracy: 0.7140 - val_loss: 1.0211 - val_accuracy: 0.6108
Epoch 2/9
63/63 [==============================] - 35s 546ms/step - loss: 0.3700 - accuracy: 0.8490 - val_loss: 1.1269 - val_accuracy: 0.5948
Epoch 3/9
63/63 [==============================] - 33s 522ms/step - loss: 0.3082 - accuracy: 0.8839 - val_loss: 1.0546 - val_accuracy: 0.6507
Epoch 4/9
63/63 [==============================] - 34s 539ms/step - loss: 0.2674 - accuracy: 0.8959 - val_loss: 1.2849 - val_accuracy: 0.6347
Epoch 5/9
63/63 [==============================] - 34s 529ms/step - loss: 0.2323 - accuracy: 0.9063 - val_loss: 1.2901 - val_accuracy: 0.6547
Epoch 6/9
63/63 [==============================] - 32s 502ms/step - loss: 0.1951 - accuracy: 0.9233 - val_loss: 1.2130 - val_accuracy: 0.6467
Epo